[WIP] Project: https://realpython.com/pandas-project-gradebook/ 

### Loading the data

In [ ]:
from pathlib import Path
import pandas as pd
import os
HERE = Path('__file__').parent
print(HERE)
DATA_FOLDER = HERE / "gradebook/data"
print(DATA_FOLDER)

In [ ]:
roster = pd.read_csv(DATA_FOLDER / "roster.csv")
roster.head()

In [ ]:
roster = pd.read_csv(
    DATA_FOLDER / "roster.csv",
    converters={"NetID": str.lower, "Email Address": str.lower},
    usecols=["Section", "Email Address", "NetID"],
    index_col="NetID",
)
roster.head()

In [ ]:
hw_exam_grades = pd.read_csv(DATA_FOLDER / "hw_exam_grades.csv")
hw_exam_grades.columns

In [ ]:
hw_exam_grades = pd.read_csv(DATA_FOLDER / "hw_exam_grades.csv",
  converters={"SID": str.lower},
  usecols=lambda x: "Submission Time" not in x,
  index_col="SID")
hw_exam_grades.head()

In [ ]:
quiz_grades = pd.DataFrame()
for file_path in DATA_FOLDER.glob("quiz_*_grades.csv"):    # https://docs.python.org/3/library/glob.html                                 
    quiz_name = "".join(file_path.stem.title().split("_")[:2]) # file_path.stem: Final path component without its suffix
    # print(file_path.stem, quiz_name)                    
    quiz = pd.read_csv(file_path, 
                      converters={"Email": str.lower},
                      index_col=["Email"],
                      usecols=["Email", "Grade"]).rename(columns={"Grade": quiz_name})
    quiz_grades = pd.concat([quiz_grades, quiz], axis=1)
quiz_grades

### Merging DataFrames

In [ ]:
# Merge DataFrame or named Series objects with a database-style join.
# Default join is inner i.e to use intersection of keys from both frames
data = pd.merge(
    roster,
    hw_exam_grades,
    left_index=True,       # Use the index from the left DataFrame as the join key(s)
    right_index=True,      # Use the index from the right DataFrame as the join key
)

In [ ]:
data

In [ ]:
final_data = pd.merge(data, quiz_grades, left_on="Email Address", right_index=True)
final_data

In [ ]:
final_data = final_data.fillna(0)
final_data

### Calculating Grades